In [1]:
def printx(listx):
    for x in listx:
        print(x)

def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list


def parse_row(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    # return Element(name, prevElemCombos)
    prevElemCombos.sort()
    for p in prevElemCombos:
        p.sort()
    return [prevElemCombo + [name] for prevElemCombo in prevElemCombos]

def parse_rows(csv_list):
    r = []
    for row in csv_list:
        r.extend(parse_row(row))
    return r


In [2]:
elemsFilename = 'combinations_list_orig.csv'
csv_list = load_csv(elemsFilename)

In [3]:
import pandas as pd

def create_elements_df(csv_list):
    r = parse_rows(csv_list)
    df = pd.DataFrame(r, columns=['elem1','elem2', 'result'])
    base_elements = get_base_elements(csv_list)
    return df[~df.index.isin(base_elements.index)]


def get_base_elements(csv_list):
    df = pd.DataFrame(parse_rows(csv_list), columns=['elem1','elem2', 'result'])
    return df[df['elem1']=='BASE_ELEMENT']

def get_elements_list(dfs):
    r = []
    for col in list(dfs.columns):
        r.extend(dfs[col].tolist())
    return list(set(r))


dfs = create_elements_df(csv_list)
dfs.head()

,elem1,elem2,result
0,rain,smog,acid rain
1,rain,smoke,acid rain
3,bird,metal,airplane
4,bird,steel,airplane
5,clock,sound,alarm clock


In [4]:
base_elements = get_base_elements(csv_list)['elem2'].tolist()

['air', 'earth', 'fire', 'water']

In [5]:
def get_unique_current_combos(curr_elements, dfs):
    possible_combos = [[elem1, elem2] for elem1 in curr_elements for elem2 in curr_elements if elem1 != elem2]
    for i in range(len(possible_combos)):
        possible_combos[i].sort()
        possible_combos[i] = '+'.join(possible_combos[i])
    possible_combos = list(set(possible_combos))
    possible_combos = [p.split('+') for p in possible_combos]
    return possible_combos
  

[['earth', 'fire'],
 ['fire', 'water'],
 ['earth', 'water'],
 ['air', 'earth'],
 ['air', 'water'],
 ['air', 'fire']]

In [6]:
def get_elem_score(elem, dfs, levels=0):
    dFilter = (dfs['elem1']==elem) | (dfs['elem2']==elem)
    if levels == 0:
        return sum(dFilter)
    else:
        next_possible_elements = dfs.loc[dFilter, 'result'].tolist()
        return sum([get_elem_score(e, dfs, levels-1) for e in next_possible_elements])

def get_next_possible_combos(curr_elements, dfs):
    r = []
    unique_combos = get_unique_current_combos(curr_elements, dfs)
    for combo in unique_combos:
        l = (dfs['elem1']==combo[0]) & (dfs['elem2']==combo[1])
        if l.any():
            r.append([dfs.loc[l, 'elem1'].tolist()[0], dfs.loc[l, 'elem2'].tolist()[0] , dfs.loc[l, 'result'].tolist()[0]])
    return [x for x in r if x[-1] not in curr_elements]

def get_next_possible_elements(curr_elements, dfs):
    r = get_next_possible_combos(curr_elements, dfs)
    return [x[-1] for x in r]


In [7]:
from joblib import Parallel, delayed
from tqdm import tqdm

def get_next_best_move(curr_elements, dfs, levels=0):
    next_possible_combos = get_next_possible_combos(curr_elements, dfs)
    scores = [get_elem_score(e[-1], dfs, levels) for e in next_possible_combos]
    return next_possible_combos[scores.index(max(scores))]

def fullCombo_to_string(combo):
    return ' + '.join(combo[:-1]) + ' = ' + combo[-1]

In [8]:
base_elements = get_base_elements(csv_list)['elem2'].tolist()
curr_elements = base_elements
base_elements

['air', 'earth', 'fire', 'water']

In [9]:
for I in range(25):
    newCombo = get_next_best_move(curr_elements, dfs, levels=3)
    print(I, fullCombo_to_string(newCombo))
    curr_elements.append(newCombo[-1])
    curr_elements.sort()


1 air + fire = energy
2 earth + water = mud
3 air + water = rain
4 earth + rain = plant
5 mud + plant = swamp
6 energy + swamp = life
7 earth + life = human
8 human + plant = farmer
9 earth + plant = grass
10 earth + fire = lava
11 air + lava = stone
12 air + stone = sand
13 fire + stone = metal
14 fire + sand = glass
15 human + metal = tool
16 energy + metal = electricity
17 life + stone = egg
18 air + energy = wind
19 earth + farmer = field
20 air + life = bird
21 energy + water = steam
22 air + steam = cloud
23 air + cloud = sky
24 fire + sky = sun
25 sun + tool = solar cell


In [10]:
def get_n_moves(curr_elements, dfs, levels=0, n=10):
    r = []
    for i in range(n):
        newCombo = get_next_best_move(curr_elements, dfs, levels=3)
        r.append(fullCombo_to_string(newCombo))
        curr_elements.append(newCombo[-1])
        curr_elements.sort()
    return r

SyntaxError: invalid syntax (3019949423.py, line 1)

# **Complete example**


In [ ]:
dfs = create_elements_df(csv_list)
base_elements = get_base_elements(csv_list)['elem2'].tolist()

get_n_moves(base_elements, dfs, levels=2, n=20)

----------------------------------------

In [ ]:
get_elem_score('energy', dfs, 3)

1067

In [ ]:
dfs[(dfs['elem1']=='fire') & (dfs['elem2']=='water')]

,elem1,elem2,result
819,fire,water,steam


In [ ]:
(dfs['elem1']=='fire') & (dfs['elem2']=='water')

0      False
1      False
3      False
4      False
5      False
       ...  
986    False
987    False
988    False
989    False
990    False
Length: 987, dtype: bool

In [ ]:
set(['fire', 'water']).symmetric_difference(set(['water', 'air']))

{'air', 'fire'}